# Paper: Hydrogen bonding in ionic liquids

#### Submission to the collection "Ionic liquids: properties and applications" in Frontiers Chemistry

Based on Samuel Tan's thesis chapter 4; Gas calculation vs. solvated (CPCM) geometries. Begin with a discussion on the optimised geometries comparing gas & CPCM, then the energetics. We need SRS-MP2 data as well as CCSD(T)/CBS.

The _spin ratio_ ($\varepsilon_s$) is defined as the opposite spin interaction energy, divided by the same-spin interaction energy.

* If $\varepsilon_s$ > 1, use coefficients: C<sub>OS</sub> = 1.640 No C<sub>SS</sub> (VTZ); C<sub>OS</sub> = 1.689 (VQZ)
* If $\varepsilon_s$ < 1, use coefficients: C<sub>OS</sub> = 0.660, C<sub>SS</sub> = 1.140 (VTZ); C<sub>OS</sub> = 0.671, C<sub>SS</sub> = 1.119 (VQZ)

In [1]:
import os
import re, sys
import numpy as np
import pandas as pd

In [2]:
srs = pd.read_csv("out_energies.csv")

In [3]:
srs.head()

,Path,File,E_Tot,E_Ref,E_Corr,E_SS,E_OS
0,./opt-HBIL-structures/srs-mp2-DMEA/,DMEA-mSO3-p1-opt-frag1.out,[H],-661.856118,-1.089066,-0.284884,-0.804182
1,./opt-HBIL-structures/srs-mp2-DMEA/,DMEA-mSO3-p1-opt-all.out,[H],-874.789049,-2.029317,-0.494869,-1.534448
2,./opt-HBIL-structures/srs-mp2-DMEA/,DMEA-mSO3-p1-opt-frag0.out,[H],-212.761421,-0.922344,-0.201100,-0.721244
3,./opt-HBIL-structures/srs-mp2-DMEA/,DMEA-Cl-p1-opt-frag1.out,[H],-459.564667,-0.196919,-0.049367,-0.147552
4,./opt-HBIL-structures/srs-mp2-DMEA/,DMEA-NO3-p1-opt-all.out,[H],-491.957311,-1.863574,-0.461067,-1.402507


In [4]:
# Split the 'File' column into System and Fragment columns:
rename = srs['File'].str.partition('opt', expand=True)
rename["System"] = rename[0] + rename[1]
rename["File"] =  rename[2].apply(lambda x: x.split('-')[1])
rename.head()

,0,1,2,System,File
0,DMEA-mSO3-p1-,opt,-frag1.out,DMEA-mSO3-p1-opt,frag1.out
1,DMEA-mSO3-p1-,opt,-all.out,DMEA-mSO3-p1-opt,all.out
2,DMEA-mSO3-p1-,opt,-frag0.out,DMEA-mSO3-p1-opt,frag0.out
3,DMEA-Cl-p1-,opt,-frag1.out,DMEA-Cl-p1-opt,frag1.out
4,DMEA-NO3-p1-,opt,-all.out,DMEA-NO3-p1-opt,all.out


In [5]:
srs["System"] = rename["System"]
srs["File"] = rename["File"]
srs["Optimisation"] = srs['Path'].str.split('/', expand=True)[1]
srs["Method"] = srs['Path'].str.split('/', expand=True)[2]

In [6]:
srs.columns.values

array(['Path', 'File', 'E_Tot', 'E_Ref', 'E_Corr', 'E_SS', 'E_OS',
       'System', 'Optimisation', 'Method'], dtype=object)

In [7]:
srs = srs[['Optimisation', 'Method', 'System', 'File', 'E_Tot', 'E_Ref', 'E_Corr', 'E_SS', 'E_OS']]

In [8]:
srs['Method'].unique()

array(['srs-mp2-DMEA', 'srs-mp2-EtMeNH2', 'srs-mp2-EtNH3', 'srs-mp2-mim',
       'srs-mp2-mpyr', 'srs-mp2-TMEA', 'hf-aVQZ-DEMA', 'hf-aVQZ-EtMeNH2',
       'hf-aVQZ-EtNH3', 'hf-aVQZ-mim', 'hf-aVQZ-mpyr', 'hf-aVQZ-TMEA',
       'mp2-VQZ-DMEA', 'mp2-VQZ-EtMeNH2', 'mp2-VQZ-EtNH3', 'mp2-VQZ-mim',
       'mp2-VQZ-mpyr', 'mp2-VQZ-TMEA'], dtype=object)

### Note: srs-mp2 in 'Method' refers to MP2/cc-pVTZ calculations.

In [9]:
# Remove the Cation from the Method column:
srs["Method"] = srs["Method"].apply(lambda x: '-'.join(x.split('-')[0:2]))

In [10]:
srs.head(5)

,Optimisation,Method,System,File,E_Tot,E_Ref,E_Corr,E_SS,E_OS
0,opt-HBIL-structures,srs-mp2,DMEA-mSO3-p1-opt,frag1.out,[H],-661.856118,-1.089066,-0.284884,-0.804182
1,opt-HBIL-structures,srs-mp2,DMEA-mSO3-p1-opt,all.out,[H],-874.789049,-2.029317,-0.494869,-1.534448
2,opt-HBIL-structures,srs-mp2,DMEA-mSO3-p1-opt,frag0.out,[H],-212.761421,-0.922344,-0.201100,-0.721244
3,opt-HBIL-structures,srs-mp2,DMEA-Cl-p1-opt,frag1.out,[H],-459.564667,-0.196919,-0.049367,-0.147552
4,opt-HBIL-structures,srs-mp2,DMEA-NO3-p1-opt,all.out,[H],-491.957311,-1.863574,-0.461067,-1.402507


In [11]:
test = srs.loc[srs["Method"] == "srs-mp2"]
# Subset where the only method used was MP2/Cc-pvtz

grouptest = test.sort_values(["Optimisation", "Method", "System", "File"])

In [12]:
aaa = grouptest.set_index(["Optimisation", "Method", "System", "File"]).sort_index()[['E_Corr']]
aaa.head()

E_Corr
Optimisation           Method  System                File               
gasopt-HBIL-structures srs-mp2 DMEA-CF3SO3-p1-gasopt all.out   -2.754867
                                                     frag0.out -0.923202
                                                     frag1.out -1.808230
                               DMEA-Cl-p1-gasopt     all.out   -1.146458
                                                     frag0.out -0.926710

In [13]:
# Calculate interaction correlation energy, OS IE, and SS IE and add as new columns:
def calcIT(grp):
    grp.sort_values(["Optimisation", "Method", "System", "File"])
    ref_corr = grp.loc[grp['File'].str.startswith('frag'), 'E_Corr']
    ref_OS = grp.loc[grp['File'].str.startswith('frag'), 'E_OS']
    ref_SS = grp.loc[grp['File'].str.startswith('frag'), 'E_SS']
    grp.loc[grp['File'].str.startswith('all'), 'IE_Corr'] = grp['E_Corr'] - (ref_corr.values[0] + ref_corr.values[1])
    grp.loc[grp['File'].str.startswith('all'), 'IE_OS'] = grp['E_OS'] - (ref_OS.values[0] + ref_OS.values[1])
    grp.loc[grp['File'].str.startswith('all'), 'IE_SS'] = grp['E_SS'] - (ref_SS.values[0] + ref_SS.values[1])
    return grp

In [14]:
srsmp2 = grouptest.groupby(['Optimisation', 'Method', 'System']).apply(calcIT) # Apply the function above
srsmp2.loc[srsmp2['System'] == 'DMEA-Cl-p1-opt'] # Quick check to see if matches hand-calculated values (it does)

,Optimisation,Method,System,File,E_Tot,E_Ref,E_Corr,E_SS,E_OS,IE_Corr,IE_OS,IE_SS
14,opt-HBIL-structures,srs-mp2,DMEA-Cl-p1-opt,all.out,[H],-672.502642,-1.136665,-0.258410,-0.878254,-0.017519,-0.009569,-0.007951
11,opt-HBIL-structures,srs-mp2,DMEA-Cl-p1-opt,frag0.out,[H],-212.761595,-0.922226,-0.201093,-0.721133,NaN,NaN,NaN
3,opt-HBIL-structures,srs-mp2,DMEA-Cl-p1-opt,frag1.out,[H],-459.564667,-0.196919,-0.049367,-0.147552,NaN,NaN,NaN


In [15]:
ecorrsrs = srsmp2.dropna()
ecorrsrs.head()

,Optimisation,Method,System,File,E_Tot,E_Ref,E_Corr,E_SS,E_OS,IE_Corr,IE_OS,IE_SS
420,gasopt-HBIL-structures,srs-mp2,DMEA-CF3SO3-p1-gasopt,all.out,[H],-1171.465425,-2.754867,-0.702029,-2.052838,-0.023435,-0.012173,-0.011262
418,gasopt-HBIL-structures,srs-mp2,DMEA-Cl-p1-gasopt,all.out,[H],-672.497149,-1.146458,-0.262571,-0.883887,-0.022828,-0.011898,-0.010930
429,gasopt-HBIL-structures,srs-mp2,DMEA-NO3-p1-gasopt,all.out,[H],-491.953965,-1.871625,-0.463870,-1.407756,-0.011511,-0.004415,-0.007096
424,gasopt-HBIL-structures,srs-mp2,DMEA-TFA-p1-gasopt,all.out,[H],-736.970606,-2.468754,-0.621076,-1.847677,-0.014888,-0.006623,-0.008265
425,gasopt-HBIL-structures,srs-mp2,DMEA-mOSO3-p1-gasopt,all.out,[H],-949.670006,-2.267871,-0.560387,-1.707485,-0.021862,-0.010953,-0.010909


In [16]:
# Creates list of deltaE values
deltaE = pd.DataFrame(srsmp2["IE_OS"]/srsmp2["IE_SS"])
deltaE.dropna().head()
deltaE.describe()

# Do two methods: treat all as deltaE > 1; separate into two groups and treat with 
# different coefficients; and compare the difference.

,0
count,79.000000
mean,1.046087
std,0.190928
min,0.349790
25%,1.009700
50%,1.095426
75%,1.164777
max,1.299736


In [17]:
np.alltrue(deltaE.dropna() > 1) 
# Lots of different delta E values! Use a Map to separate into the two vals.

False

### SRS-MP2 scaling: using MP2/cc-pVTZ coefficients
Scale according to the equation:
$E_{SRS-MP2}^{Correlation-energy}$= C$_{OS}$E$_{OS}$ + C$_{SS}$E$_{SS}$
* if $\Delta\varepsilon_{s} < 1$, C$_{OS} = 0.660$ and C$_{SS}= 1.140$ <br>
* if $\Delta\varepsilon_{s} \geq 1$, $C_{OS} = 1.640$.

In [18]:
# First, treat all using the >= 1 coefficients only (drop SS values) 
srsmp2['SRS-MP2-IEcorr'] = 1.640 * srsmp2['IE_OS']

In [19]:
ecorr = srsmp2.dropna()
ecorr.head()

,Optimisation,Method,System,File,E_Tot,E_Ref,E_Corr,E_SS,E_OS,IE_Corr,IE_OS,IE_SS,SRS-MP2-IEcorr
420,gasopt-HBIL-structures,srs-mp2,DMEA-CF3SO3-p1-gasopt,all.out,[H],-1171.465425,-2.754867,-0.702029,-2.052838,-0.023435,-0.012173,-0.011262,-0.019963
418,gasopt-HBIL-structures,srs-mp2,DMEA-Cl-p1-gasopt,all.out,[H],-672.497149,-1.146458,-0.262571,-0.883887,-0.022828,-0.011898,-0.010930,-0.019513
429,gasopt-HBIL-structures,srs-mp2,DMEA-NO3-p1-gasopt,all.out,[H],-491.953965,-1.871625,-0.463870,-1.407756,-0.011511,-0.004415,-0.007096,-0.007241
424,gasopt-HBIL-structures,srs-mp2,DMEA-TFA-p1-gasopt,all.out,[H],-736.970606,-2.468754,-0.621076,-1.847677,-0.014888,-0.006623,-0.008265,-0.010862
425,gasopt-HBIL-structures,srs-mp2,DMEA-mOSO3-p1-gasopt,all.out,[H],-949.670006,-2.267871,-0.560387,-1.707485,-0.021862,-0.010953,-0.010909,-0.017962


In [121]:
HFenergies = srs.loc[srs["Method"] == "hf-aVQZ"].drop(columns=['E_Ref', 'E_Corr', 'E_SS', 'E_OS'])

In [122]:
HFenergies.head()

,Optimisation,Method,System,File,E_Tot
126,opt-HBIL-structures,hf-aVQZ,DMEA-CF3SO3-p1-opt,all.out,-1171.5452325037424544
127,opt-HBIL-structures,hf-aVQZ,DMEA-CF3SO3-p1-opt,frag0.out,-212.7782484674152670
128,opt-HBIL-structures,hf-aVQZ,DMEA-CF3SO3-p1-opt,frag1.out,-958.6237429393129332
129,opt-HBIL-structures,hf-aVQZ,DMEA-Cl-p1-opt,all.out,-672.5223342919188099
130,opt-HBIL-structures,hf-aVQZ,DMEA-Cl-p1-opt,frag0.out,-212.7757994809106776


In [123]:
def calcHF_IE(grp):
    grp.sort_values(["Optimisation", "Method", "System", "File"])
    ref = grp.loc[grp['File'].str.startswith('frag'), 'E_Tot']
    grp.loc[grp['File'].str.startswith('all'), 'IE_HF'] = grp['E_Tot'].astype(float) - (float(ref.values[0]) + float(ref.values[1]))
    return grp

In [124]:
intHF = HFenergies.groupby(['Optimisation', 'Method', 'System']).apply(calcHF_IE)

In [125]:
HF_interactions = intHF.dropna()

In [138]:
totalSRSallOS = pd.merge(HF_interactions, ecorr, how='inner', on=['Optimisation', 'System'])
totalSRSallOS.head()

,Optimisation,Method_x,System,File_x,E_Tot_x,IE_HF,Method_y,File_y,E_Tot_y,E_Ref,E_Corr,E_SS,E_OS,IE_Corr,IE_OS,IE_SS,SRS-MP2-IEcorr
0,opt-HBIL-structures,hf-aVQZ,DMEA-CF3SO3-p1-opt,all.out,-1171.5452325037424544,-0.143241,srs-mp2,all.out,[H],-1171.466855,-2.745676,-0.697928,-2.047749,-0.016672,-0.008835,-0.007838,-0.014489
1,opt-HBIL-structures,hf-aVQZ,DMEA-Cl-p1-opt,all.out,-672.5223342919188099,-0.170182,srs-mp2,all.out,[H],-672.502642,-1.136665,-0.258410,-0.878254,-0.017519,-0.009569,-0.007951,-0.015692
2,opt-HBIL-structures,hf-aVQZ,DMEA-mOSO3-p1-opt,all.out,-949.7361375039931772,-0.159520,srs-mp2,all.out,[H],-949.676531,-2.259516,-0.556711,-1.702805,-0.016550,-0.008478,-0.008072,-0.013903
3,opt-HBIL-structures,hf-aVQZ,DMEA-mSO3-p1-opt,all.out,-874.8417925007586291,-0.166493,srs-mp2,all.out,[H],-874.789049,-2.029317,-0.494869,-1.534448,-0.017907,-0.009022,-0.008885,-0.014796
4,opt-HBIL-structures,hf-aVQZ,DMEA-NO3-p1-opt,all.out,-491.9959346590015912,-0.163615,srs-mp2,all.out,[H],-491.957311,-1.863574,-0.461067,-1.402507,-0.021105,-0.011644,-0.009462,-0.019095


In [141]:
totalSRSallOS = totalSRSallOS[['Optimisation','System', 'File_x', 'IE_HF', 'SRS-MP2-IEcorr']]

In [142]:
totalSRSallOS['TotalIE'] = totalSRSallOS['IE_HF'] + totalSRSallOS['SRS-MP2-IEcorr']

In [144]:
totalSRSallOS['IE_HF_kjmol']  = totalSRSallOS['IE_HF'] * 2625.5
totalSRSallOS['IE_Corr_kjmol']  = totalSRSallOS['SRS-MP2-IEcorr'] * 2625.5
totalSRSallOS['IE_tot_kjmol']  = totalSRSallOS['TotalIE'] * 2625.5
totalSRSallOS.describe()

,IE_HF,SRS-MP2-IEcorr,TotalIE,IE_HF_kjmol,IE_Corr_kjmol,IE_tot_kjmol
count,79.000000,79.000000,79.000000,79.000000,79.000000,79.000000
mean,-0.135973,-0.014016,-0.149989,-356.997989,-36.799291,-393.797280
std,0.049488,0.004475,0.052292,129.931798,11.750001,137.293185
min,-0.253486,-0.022206,-0.266047,-665.527045,-58.300726,-698.507109
25%,-0.166083,-0.017524,-0.182950,-436.050562,-46.008268,-480.335257
50%,-0.146563,-0.014489,-0.160712,-384.801013,-38.040854,-421.948044
75%,-0.128704,-0.011107,-0.141374,-337.912331,-29.161629,-371.178178
max,-0.013347,-0.002294,-0.019759,-35.042671,-6.023721,-51.876536


#### Treating with different coefficients based on DeltaE values:

In [158]:
# Add column of spin ratio to srsmp2 dataframe by using a mask:
srsmp2["deltaE"] = srsmp2["IE_OS"]/srsmp2["IE_SS"]

mask1 = srsmp2['deltaE'] < 1
lessthan1 = srsmp2[srsmp2['deltaE'] < 1].copy() # Contains 19 entries
geq1 = srsmp2[srsmp2['deltaE'] >= 1].copy() # Contains 60 entries

In [166]:
# Add the new SRS interaction energy column to each df then merge the two together:
geq1['SRS-MP2-IEcorr'] = 1.640 * geq1['IE_OS']
lessthan1['SRS-MP2-IEcorr'] = 0.660 * lessthan1['IE_OS'] + 1.140 * lessthan1['IE_SS']

mixed = pd.concat([geq1, lessthan1])
mixed.tail(2)

,Optimisation,Method,System,File,E_Tot,E_Ref,E_Corr,E_SS,E_OS,IE_Corr,IE_OS,IE_SS,SRS-MP2-IEcorr,deltaE
80,opt-HBIL-structures,srs-mp2,mim-TFA-p1-opt,all.out,[H],-788.506687,-2.577254,-0.672494,-1.904760,-0.006936,-0.003182,-0.003754,-0.006380,0.847417
63,opt-HBIL-structures,srs-mp2,mim-mSO3-p1-opt,all.out,[H],-926.345262,-2.147643,-0.550947,-1.596696,-0.012190,-0.005510,-0.006680,-0.011252,0.824856


In [173]:
# Now add in the HF values and calculate total IE:
totalSRSmixed = pd.merge(HF_interactions, mixed, how='inner', on=['Optimisation', 'System'])

totalSRSmixed = totalSRSmixed[['Optimisation','System', 'File_x', 'IE_HF', 'SRS-MP2-IEcorr']]
totalSRSmixed['TotalIE'] = totalSRSmixed['IE_HF'] + totalSRSmixed['SRS-MP2-IEcorr']

totalSRSmixed['IE_HF_kjmol']  = totalSRSmixed['IE_HF'] * 2625.5
totalSRSmixed['IE_Corr_kjmol']  = totalSRSmixed['SRS-MP2-IEcorr'] * 2625.5
totalSRSmixed['IE_tot_kjmol']  = totalSRSmixed['TotalIE'] * 2625.5
totalSRSmixed.describe()

,IE_HF,SRS-MP2-IEcorr,TotalIE,IE_HF_kjmol,IE_Corr_kjmol,IE_tot_kjmol
count,79.000000,79.000000,79.000000,79.000000,79.000000,79.000000
mean,-0.135973,-0.014639,-0.150612,-356.997989,-38.433415,-395.431403
std,0.049488,0.003878,0.051456,129.931798,10.180934,135.097964
min,-0.253486,-0.022464,-0.267913,-665.527045,-58.979287,-703.406215
25%,-0.166083,-0.017613,-0.182950,-436.050562,-46.243653,-480.335257
50%,-0.146563,-0.014559,-0.160712,-384.801013,-38.225425,-421.948044
75%,-0.128704,-0.012533,-0.141374,-337.912331,-32.904373,-371.178178
max,-0.013347,-0.005483,-0.022459,-35.042671,-14.394857,-58.964921


#### Now calculating: MP2/cc-pVQZ SRS values!! $C_{OS}$ = 0.671 and $C_{SS}$ = 1.119

In [174]:
# To do: MP2/cc-pVQZ SRS values!! C_OS = 0.671 and C_SS = 1.119
vqz = srs.loc[srs["Method"] == "mp2-VQZ"]
vqz = vqz.sort_values(["Optimisation", "Method", "System", "File"])

In [189]:
srsvqz = vqz.groupby(['Optimisation', 'Method', 'System']).apply(calcIT)
srsvqz.head()

,Optimisation,Method,System,File,E_Tot,E_Ref,E_Corr,E_SS,E_OS,IE_Corr,IE_OS,IE_SS
624,gasopt-HBIL-structures,mp2-VQZ,DMEA-CF3SO3-p1-gasopt,all.out,[H],-1171.539596,-2.971742,-0.734319,-2.237423,-0.021496,-0.010828,-0.010667
625,gasopt-HBIL-structures,mp2-VQZ,DMEA-CF3SO3-p1-gasopt,frag0.out,[H],-212.771007,-0.980058,-0.208137,-0.771921,NaN,NaN,NaN
626,gasopt-HBIL-structures,mp2-VQZ,DMEA-CF3SO3-p1-gasopt,frag1.out,[H],-958.615118,-1.970188,-0.515515,-1.454673,NaN,NaN,NaN
627,gasopt-HBIL-structures,mp2-VQZ,DMEA-Cl-p1-gasopt,all.out,[H],-672.515226,-1.226216,-0.272445,-0.953771,-0.021699,-0.010923,-0.010776
628,gasopt-HBIL-structures,mp2-VQZ,DMEA-Cl-p1-gasopt,frag0.out,[H],-212.749322,-0.983566,-0.209121,-0.774445,NaN,NaN,NaN


In [186]:
# Applying > 1 values to all:
srsvqz['SRS-MP2-IEcorr'] = 1.689 * srsvqz['IE_OS']

In [178]:
vals = srsvqz.dropna()

In [187]:
totalSRSvqz = pd.merge(HF_interactions, vals, how='inner', on=['Optimisation', 'System'])
totalSRSvqz = totalSRSvqz[['Optimisation','System', 'File_x', 'IE_HF', 'SRS-MP2-IEcorr']]
totalSRSvqz['TotalIE'] = totalSRSvqz['IE_HF'] + totalSRSvqz['SRS-MP2-IEcorr']
totalSRSvqz['SRS-MP2-IEcorr']
totalSRSvqz.head()

,Optimisation,System,File_x,IE_HF,SRS-MP2-IEcorr,TotalIE
0,opt-HBIL-structures,DMEA-CF3SO3-p1-opt,all.out,-0.143241,-0.013376,-0.156617
1,opt-HBIL-structures,DMEA-Cl-p1-opt,all.out,-0.170182,-0.015118,-0.185300
2,opt-HBIL-structures,DMEA-mOSO3-p1-opt,all.out,-0.159520,-0.012899,-0.172419
3,opt-HBIL-structures,DMEA-mSO3-p1-opt,all.out,-0.166493,-0.013584,-0.180077
4,opt-HBIL-structures,DMEA-NO3-p1-opt,all.out,-0.163615,-0.017223,-0.180837
